In [1]:
import numpy as np
import pyBigWig
import pysam
import os

In [2]:
class Density:
    def values(self, chrom, start, end, strand):
        return 0


class ReadDensity(Density):
    """
    ReadDensity class. See 
    https://github.com/byee4/rbp-maps/maps/density/ReadDensity.py 
    for a real example
    
    Attributes:
        self.pos(positive *.bw file)
        self.neg(negative *.bw file)
    """

    def __init__(self, pos, neg, name=None, bam=None):
        try:
            self.pos = pyBigWig.open(pos)
            self.neg = pyBigWig.open(neg)
        except Exception as e:
            print("couldn't open the bigwig files!")
            print(e)

    def values(self, chrom, start, end, strand):
        """
        Parameters
        ----------
        chrom : basestring
            (eg. chr1)
        start : int
            0-based start (first position in chromosome is 0)
        end : int
            1-based end (last position is not included)
        strand : str
            either '+' or '-'
        Returns
        -------
        densites : list
            values corresponding to density over specified positions.
        """

        try:
            if strand == "+":
                return self.pos.values(chrom, start, end)
            elif strand == "-":
                return list(reversed(self.neg.values(chrom, start, end)))
            else:
                print("Strand neither + or -")
                return 1
        except RuntimeError:
            # usually occurs when no chromosome exists in the bigwig file
            return [np.NaN] * abs(start - end)

In [3]:

rdd = ReadDensity(
    pos='/projects/ps-yeolab3/encode/analysis/encode_master/218_01_TRA2A.merged.r2.norm.pos.bw',
    neg='/projects/ps-yeolab3/encode/analysis/encode_master/218_01_TRA2A.merged.r2.norm.neg.bw'
)

rdd.values(
    chrom='chr2',
    start=63221025,
    end=63221092,
    strand='+'
)

[10.137833595275879,
 10.137833595275879,
 10.137833595275879,
 10.137833595275879,
 8.110267639160156,
 8.110267639160156,
 8.110267639160156,
 7.0964837074279785,
 6.082700252532959,
 6.082700252532959,
 6.082700252532959,
 6.082700252532959,
 7.0964837074279785,
 7.0964837074279785,
 7.0964837074279785,
 7.0964837074279785,
 7.0964837074279785,
 8.110267639160156,
 9.12405014038086,
 9.12405014038086,
 9.12405014038086,
 9.12405014038086,
 12.165400505065918,
 12.165400505065918,
 12.165400505065918,
 13.179183959960938,
 9.12405014038086,
 9.12405014038086,
 10.137833595275879,
 10.137833595275879,
 10.137833595275879,
 12.165400505065918,
 13.179183959960938,
 13.179183959960938,
 13.179183959960938,
 13.179183959960938,
 13.179183959960938,
 14.192967414855957,
 14.192967414855957,
 15.206750869750977,
 15.206750869750977,
 13.179183959960938,
 13.179183959960938,
 12.165400505065918,
 12.165400505065918,
 12.165400505065918,
 13.179183959960938,
 14.192967414855957,
 17.23431777